# Simple Entry Removal Algorithm 1 (SER 1)

In [13]:
import numpy as np
import datetime
import pandas as pd
import json

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [7]:
def add_up_a_decomposition(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=int)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# The SER 1 algorithm

In [5]:
# This function is used in the SER 1 function.
# residue_matrix_R is a np.ndarray.
# PBN_row_num is the number of rows present in residue_matrix_R.
# PBN_col_num is the number of columns present in residue_matrix_R.
def find_smallest_nonzero_entry(residue_matrix_R, PBN_row_num, PBN_col_num):
    row_counter = 0
    col_counter = 0
    while residue_matrix_R[row_counter, col_counter] == 0:
        row_counter += 1
    smallest_entry = residue_matrix_R[row_counter, col_counter]
    smallest_entry_row_pos = row_counter
    smallest_entry_col_pos = col_counter

    del row_counter
    del col_counter
    for col_counter in range(PBN_col_num):
        for row_counter in range(PBN_row_num):
            current_entry = residue_matrix_R[row_counter, col_counter]
            if (current_entry > 0) and (current_entry < smallest_entry):
                smallest_entry = current_entry
                smallest_entry_row_pos = row_counter
                smallest_entry_col_pos = col_counter

    return smallest_entry, smallest_entry_row_pos, smallest_entry_col_pos

In [6]:
def SER_1(PBN_matrix_P):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4
    chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                          PBN_row_num, PBN_col_num)
    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                              PBN_row_num, PBN_col_num)
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

### Outputs of SER 1 on different small PBN matrices

#### [1] PBN Matrix from Default Data

In [4]:
test_PBN_matrix_3 = np.array([[57,  0, 10,  0,  0,  4,  0,  0], 
                              [ 0, 15,  0,  0,  0,  8,  0,  0], 
                              [ 0,  8, 40, 25, 25,  0, 67,  0], 
                              [ 0,  0,  0,  0, 38,  0,  0,  0],
                              [14, 31,  0, 50, 12, 13, 33,  6],
                              [29, 31, 20,  0, 25, 29,  0, 39],
                              [ 0, 15, 30,  0,  0, 13,  0,  0],
                              [ 0,  0,  0, 25,  0, 33,  0, 55]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(test_PBN_matrix_3)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(20,
 [4, 6, 8, 8, 10, 12, 7, 9, 6, 7, 4, 4, 4, 2, 4, 1, 1, 1, 1, 1],
 array([[0, 4, 2, 4, 3, 0, 2, 7],
        [0, 5, 2, 4, 3, 7, 2, 4],
        [0, 2, 2, 4, 3, 5, 2, 7],
        [0, 4, 6, 4, 2, 1, 2, 7],
        [0, 5, 0, 2, 5, 7, 2, 5],
        [5, 4, 2, 7, 4, 5, 4, 7],
        [0, 4, 6, 4, 3, 7, 2, 5],
        [5, 1, 5, 4, 2, 5, 2, 7],
        [0, 1, 6, 2, 5, 4, 4, 5],
        [4, 5, 5, 7, 3, 4, 2, 5],
        [0, 6, 5, 2, 5, 6, 4, 7],
        [0, 6, 2, 4, 2, 7, 4, 7],
        [5, 5, 6, 4, 3, 6, 2, 5],
        [4, 6, 2, 7, 3, 7, 4, 7],
        [5, 6, 6, 2, 5, 6, 4, 5],
        [4, 6, 2, 7, 2, 7, 2, 7],
        [4, 5, 6, 7, 2, 7, 2, 7],
        [4, 5, 2, 2, 2, 7, 2, 7],
        [4, 5, 2, 7, 2, 6, 2, 5],
        [4, 5, 2, 7, 5, 7, 4, 7]], dtype=int64))

In [5]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=2785)

#### [2] Nursing Home PBN Matrix

In [6]:
test_PBN_matrix_4 = np.array([[0, 6, 8, 1, 7, 5, 8, 5, 6, 5, 3, 4, 10, 2, 2, 0], 
                              [8, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, 0, 0, 3], 
                              [2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, 0, 0, 0], 
                              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 1, 1, 8, 2, 4, 2, 5, 0, 3, 2, 3,  0, 3, 4, 1],
                              [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 3,  0, 0, 2, 1],
                              [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,  0, 4, 2, 5],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 1, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0]
                             ])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(test_PBN_matrix_4)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(10,
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 array([[ 1,  1,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  4, 12],
        [ 1,  2,  0,  4,  0,  0,  0,  4,  0,  0, 12,  0,  0,  4,  4, 12],
        [ 1,  3,  0,  4,  0,  4,  0,  0,  0,  0,  0,  4,  0, 12,  0,  1],
        [ 1,  4,  0,  4,  0,  0,  0,  4,  0,  4,  4,  8,  0,  0,  4, 12],
        [ 1,  0,  1,  4,  0,  4,  0,  0,  0,  0,  9,  0,  0,  4,  8,  1],
        [ 1,  0,  4,  4,  0,  0,  0,  4,  0,  4, 12,  4,  0, 12, 12, 12],
        [ 1,  0,  0,  0,  4,  4,  0,  0,  1,  9,  0,  8,  0,  0,  0,  1],
        [ 1,  0,  0,  4,  0,  0,  4,  4,  2,  0,  4,  0,  0,  4,  4,  4],
        [ 2,  0,  0,  4,  4,  4,  0,  0,  8,  4,  9,  4,  0, 12,  8,  8],
        [ 2,  0,  0,  5,  8,  5,  4,  4,  9,  9, 12,  8,  0, 13, 12, 12]],
       dtype=int64))

In [7]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1004)

#### [3] First Synthetic PBN Matrix

In [8]:
test_PBN_matrix_5_permuted = np.array([[12, 30, 22, 10, 10, 15, 54, 34],
                                       [10, 24, 19, 54, 30,  0,  0,  0],
                                       [54, 15,  0, 12, 12,  0,  0, 30],
                                       [ 0,  0, 24, 15, 24, 19, 10,  0],
                                       [ 0,  0,  0,  0, 34, 10, 12,  0],
                                       [19,  0, 15,  0,  0, 12,  0, 22],
                                       [15, 19,  0, 19,  0, 54,  0, 24],
                                       [ 0, 22, 30,  0,  0,  0, 34,  0]])

In [9]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(test_PBN_matrix_5_permuted)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(23,
 [10, 10, 10, 10, 10, 9, 10, 4, 4, 5, 5, 5, 3, 2, 2, 3, 1, 1, 2, 1, 1, 1, 1],
 array([[1, 0, 7, 1, 4, 6, 0, 0],
        [2, 1, 3, 0, 1, 6, 0, 2],
        [2, 7, 0, 1, 0, 6, 0, 0],
        [2, 0, 7, 1, 3, 4, 7, 6],
        [2, 0, 1, 1, 4, 6, 0, 5],
        [5, 6, 1, 6, 1, 3, 7, 2],
        [5, 2, 5, 3, 3, 0, 7, 0],
        [6, 1, 3, 1, 3, 6, 0, 6],
        [2, 7, 0, 2, 4, 5, 4, 0],
        [0, 2, 3, 1, 2, 3, 0, 5],
        [6, 1, 3, 6, 1, 6, 3, 2],
        [2, 1, 7, 2, 4, 5, 4, 6],
        [0, 6, 0, 2, 2, 0, 0, 5],
        [6, 7, 0, 1, 1, 0, 3, 2],
        [2, 6, 5, 3, 4, 3, 0, 6],
        [2, 7, 7, 6, 1, 6, 7, 2],
        [0, 6, 0, 1, 1, 3, 3, 5],
        [6, 6, 5, 3, 2, 5, 4, 2],
        [0, 6, 0, 1, 2, 3, 3, 5],
        [0, 7, 5, 3, 4, 5, 4, 6],
        [6, 6, 7, 6, 4, 6, 7, 6],
        [6, 7, 5, 3, 2, 5, 4, 5],
        [6, 7, 7, 6, 4, 6, 7, 6]], dtype=int64))

In [10]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1559)

#### [4] Second Synthetic PBN Matrix

In [11]:
test_PBN_matrix_6_permuted = np.array([[ 0,  0,  0, 49,  0, 43,  0, 49],
                                       [ 0, 30, 12,  0, 30,  0, 25,  0],
                                       [25,  0,  0, 15,  0, 22, 12, 15],
                                       [ 0,  0, 10, 24, 19,  0, 30,  0],
                                       [30, 43, 15,  0, 24, 15,  0,  0],
                                       [43,  0, 49,  0,  0,  0, 19, 24],
                                       [ 0, 22,  0, 22, 12, 30,  0, 12],
                                       [12, 15, 24,  0, 25,  0, 24, 10]])

In [12]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(test_PBN_matrix_6_permuted)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(23,
 [10, 10, 12, 12, 8, 8, 8, 7, 5, 5, 4, 3, 4, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1],
 array([[5, 4, 3, 0, 1, 0, 3, 0],
        [5, 4, 5, 0, 7, 0, 1, 7],
        [7, 1, 5, 0, 4, 6, 7, 0],
        [4, 4, 1, 3, 1, 0, 3, 0],
        [2, 6, 5, 6, 1, 2, 5, 5],
        [5, 1, 7, 0, 3, 6, 3, 5],
        [4, 7, 5, 2, 7, 4, 1, 5],
        [2, 7, 7, 6, 4, 2, 2, 0],
        [5, 6, 4, 3, 4, 0, 7, 2],
        [2, 4, 5, 0, 6, 6, 2, 6],
        [4, 1, 4, 0, 3, 2, 5, 2],
        [5, 6, 7, 2, 3, 2, 1, 0],
        [5, 1, 4, 2, 6, 4, 5, 6],
        [4, 1, 5, 3, 7, 0, 7, 2],
        [2, 4, 4, 6, 7, 6, 7, 0],
        [2, 6, 7, 3, 3, 0, 1, 2],
        [4, 4, 5, 6, 3, 4, 5, 0],
        [4, 4, 5, 6, 6, 0, 7, 6],
        [5, 6, 7, 6, 7, 6, 1, 0],
        [4, 4, 5, 3, 6, 4, 5, 0],
        [4, 6, 7, 6, 7, 6, 7, 6],
        [5, 4, 5, 3, 6, 4, 5, 2],
        [5, 6, 7, 6, 7, 6, 7, 6]], dtype=int64))

In [13]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1797)

#### [5] Third Synthetic PBN Matrix

In [14]:
test_PBN_matrix_7_permuted = np.array([[ 26,   0,   0,   0,  59,   0,  49,   0,  29,   0,   0,   0,   0,   9,   0,  46],
                                       [  0,   0,  39,  17,   0,   0,   0,  49,  49,   0,   0,   0,   0,  98,  54,  17],
                                       [  0,   0,  26,  49,   0,   0,   0,   9,   0,   0,   0,  59,   9,   0,   0,   0],
                                       [  0,   0,   0,   0,   0,   9,  54,   0,   0,   0,   0,   0,  39,   0,  49,  26],
                                       [ 49,   0,   9,   0,   0,  29,   0,   0,   0,   0, 108,  63,   0,  17,  59,   0],
                                       [  0,   0,   0,  26,   0,   0,   0,  17,  39,  29,   0,   0,   0,   0,   0,  88],
                                       [ 17,  63,  88,   0,   0,   0,   0,  98,   0,   9,  37,   0,  88,   0,   0,  49],
                                       [  0,   0,   0,   0,  49,   0,   0,   0,  17,  37,   9,  29,  63,  49,  39,   0],
                                       [ 98,   0,   0,   0,  46, 108,  26,   0,   0,   0,   0,   9,   0,  63,   9,   0],
                                       [  0,  29,   0,   0,  17,   0,  29,   0,  46,   0,  26,   0,   0,   0,  26,   0],
                                       [ 29, 108,   0,  88,   0,  39,   0,   0,   0,   0,  39,   0,   0,  29,   0,   0],
                                       [  0,  39,   0,  39,   0,   0,   0,   0,  26,  26,  29,  39,   0,   0,   0,   0],
                                       [  0,   0,   0,   0,  29,   0,  98,  29,   0,  39,   0,   0,   0,   0,   0,   0],
                                       [ 46,   0,  49,  37,  39,  26,   9,   0,   0, 108,  17,   0,  49,   0,  29,   0],
                                       [  0,   9,  54,   9,   0,  54,   0,  37,   0,  17,   0,  49,  17,   0,   0,  39],
                                       [  0,  17,   0,   0,  26,   0,   0,  26,  59,   0,   0,  17,   0,   0,   0,   0]])


In [15]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(test_PBN_matrix_7_permuted)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(47,
 [9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  17,
  11,
  11,
  13,
  10,
  10,
  8,
  8,
  8,
  6,
  7,
  6,
  3,
  5,
  4,
  4,
  3,
  3,
  2,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 array([[ 8, 14,  6, 10,  0,  8, 12,  6, 15, 13,  4,  4,  6,  1,  4,  5],
        [ 8, 10,  4, 10,  0,  8, 12,  6, 15, 13,  4,  2,  6,  1,  1,  5],
        [ 8, 10,  6, 14,  7,  8, 12,  6,  1, 13,  4,  4,  6,  1,  4,  5],
        [ 8, 10,  6, 10,  8,  3, 12,  6,  9, 13,  4,  2,  7,  1,  3,  5],
        [ 8, 10,  6, 10,  0,  8, 13,  6, 15, 13,  4, 14,  6,  8,  1,  5],
        [ 8, 10, 14, 10,  7,  8, 12,  2,  1, 13,  4,  4,  7,  1,  4,  6],
        [ 4,  6,  6,  2, 13,  8,  3,  6,  5,  6,  4,  2,  6,  8,  3,  0],
        [13, 10, 13, 10,  8,  8, 12,  1,  9, 13,  7, 14, 13,  1,  7,  5],
        [ 8,  6, 14,  2,  0, 14,  0,  6, 15, 13,  4,  8,  7,  7,  1,  6],
        [ 4, 10,  6, 11,  7,  8,  3,  1,  1, 12, 10, 11,  2,  8,  4, 14],
        [13, 

In [16]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=8434)

#### [6] Small Synthetic PBN Matrix

In [17]:
test_PBN_matrix_2 = np.array([[1, 5, 6, 0], 
                              [4, 0, 2, 0], 
                              [5, 2, 0, 10], 
                              [0, 3, 2, 0]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(test_PBN_matrix_2)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(7,
 [1, 2, 2, 2, 1, 1, 1],
 array([[0, 0, 0, 2],
        [2, 2, 0, 2],
        [1, 0, 1, 2],
        [1, 3, 0, 2],
        [2, 3, 3, 2],
        [2, 0, 0, 2],
        [2, 0, 3, 2]], dtype=int64))

In [18]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(0)

# Testing SER 1 on the three PBN matrices from the MOMP paper.

### 1. First PBN matrix $P_1$

In [19]:
MOMP_PBN_P1 = np.array([[1, 3, 2, 1], 
                        [2, 3, 2, 0], 
                        [0, 0, 6, 4], 
                        [7, 4, 0, 5]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(MOMP_PBN_P1)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(7,
 [1, 1, 2, 1, 1, 2, 2],
 array([[0, 3, 2, 3],
        [3, 0, 2, 0],
        [1, 1, 2, 2],
        [3, 1, 0, 3],
        [3, 3, 0, 3],
        [3, 0, 1, 2],
        [3, 3, 2, 3]], dtype=int64))

In [20]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1366)

### 2. Second PBN matrix $P_2$

In [21]:
MOMP_PBN_P2 = np.array([[1, 3, 2, 1, 0, 0, 0, 0], 
                        [2, 3, 2, 0, 0, 0, 0, 0], 
                        [0, 0, 6, 4, 0, 0, 0, 0], 
                        [7, 4, 0, 5, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 3, 2, 1],
                        [0, 0, 0, 0, 2, 3, 2, 0],
                        [0, 0, 0, 0, 0, 0, 6, 4],
                        [0, 0, 0, 0, 7, 4, 0, 5]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(MOMP_PBN_P2)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(9,
 [1, 1, 1, 1, 2, 1, 1, 1, 1],
 array([[0, 3, 2, 3, 7, 7, 6, 7],
        [3, 0, 2, 0, 7, 4, 6, 6],
        [3, 1, 2, 2, 4, 5, 6, 7],
        [3, 3, 2, 3, 7, 7, 6, 4],
        [1, 0, 0, 2, 7, 4, 4, 6],
        [3, 1, 1, 2, 5, 5, 5, 7],
        [3, 1, 2, 3, 7, 7, 6, 7],
        [3, 3, 1, 3, 5, 5, 5, 6],
        [3, 3, 2, 3, 7, 7, 6, 7]], dtype=int64))

In [22]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1332)

### 3. Third PBN matrix $P_3$

In [23]:
MOMP_PBN_P3 = np.array([[57,  0, 10,  0,  0,  4,  0,  0], 
                        [14, 31,  0, 50, 12, 13, 33,  6], 
                        [ 0,  8, 40, 25, 25,  0, 67,  0], 
                        [ 0, 15,  0,  0,  0,  8,  0,  0],
                        [ 0, 15, 30,  0,  0, 13,  0,  0],
                        [29, 31, 20,  0, 25, 29,  0, 39],
                        [ 0,  0,  0,  0, 38,  0,  0,  0],
                        [ 0,  0,  0, 25,  0, 33,  0, 55]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(MOMP_PBN_P3)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(20,
 [4, 6, 8, 8, 10, 12, 7, 9, 6, 7, 4, 4, 4, 2, 4, 1, 1, 1, 1, 1],
 array([[0, 1, 2, 1, 6, 0, 2, 7],
        [0, 5, 2, 1, 6, 7, 2, 1],
        [0, 2, 2, 1, 6, 5, 2, 7],
        [0, 1, 4, 1, 2, 3, 2, 7],
        [0, 5, 0, 2, 5, 7, 2, 5],
        [5, 1, 2, 7, 1, 5, 1, 7],
        [0, 1, 4, 1, 6, 7, 2, 5],
        [5, 3, 5, 1, 2, 5, 2, 7],
        [0, 3, 4, 2, 5, 1, 1, 5],
        [1, 4, 5, 7, 6, 1, 1, 5],
        [0, 5, 5, 2, 5, 4, 2, 7],
        [0, 5, 2, 1, 2, 7, 2, 7],
        [5, 4, 4, 1, 6, 4, 1, 5],
        [1, 5, 2, 7, 6, 7, 2, 7],
        [5, 5, 4, 2, 5, 4, 2, 5],
        [1, 5, 2, 7, 2, 7, 1, 7],
        [1, 4, 4, 7, 2, 7, 1, 7],
        [1, 4, 2, 2, 2, 7, 1, 7],
        [1, 4, 2, 7, 2, 4, 1, 5],
        [1, 4, 2, 7, 5, 7, 2, 7]], dtype=int64))

In [24]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1721)

# Testing SER 1 on the PBN matrices from the projection-based gradient descent method (PBGDM) paper.

### 1. First PBN matrix $P_1$

In [25]:
PBGDM_PBN_P1 = np.array([[1, 3, 5, 6], 
                         [0, 7, 0, 0], 
                         [0, 0, 5, 0], 
                         [9, 0, 0, 4]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P1)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(5,
 [1, 3, 2, 2, 2],
 array([[0, 1, 0, 0],
        [3, 0, 2, 0],
        [3, 1, 2, 3],
        [3, 1, 0, 0],
        [3, 1, 0, 3]], dtype=int64))

In [26]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1051)

### 2. Second PBN matrix $P_3$

In [27]:
PBGDM_PBN_P3 = np.array([[10,  0,  0, 2,  0, 0, 0,  0], 
                         [ 0,  0,  0, 2,  0, 0, 0,  0], 
                         [ 0,  0,  0, 0, 10, 0, 0,  0], 
                         [ 0,  0,  0, 0,  0, 0, 0,  0],
                         [ 0,  0,  0, 3,  0, 0, 5,  0],
                         [ 0,  0,  0, 3,  0, 0, 5,  0],
                         [ 0, 10, 10, 0,  0, 5, 0,  0],
                         [ 0,  0,  0, 0,  0, 5, 0, 10]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P3)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(4,
 [2, 2, 3, 3],
 array([[0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7],
        [0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7]], dtype=int64))

In [28]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1011)

### 3. Third PBN matrix $P_4(d)$, where $d = 0.01, 0.02, 0.03, 0.04$.

In [29]:
d = 1
PBGDM_PBN_P4_d1 = np.array([[100 - d,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0,  0, 100 - d,  0,  0,       0], 
                            [      d,       d,       d,  0,       d,  0,  0,       d],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0, 100 - d, 100 - d,  0,       0, 50,  0,       0],
                            [      0,       0,       0,  0,       0, 50,  0, 100 - d]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P4_d1)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(11,
 [1, 1, 1, 1, 1, 20, 20, 27, 1, 1, 26],
 array([[3, 6, 6, 4, 2, 6, 4, 7],
        [0, 3, 6, 5, 2, 7, 5, 7],
        [0, 6, 3, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 3, 7, 5, 7],
        [0, 6, 6, 4, 2, 6, 4, 3],
        [0, 6, 6, 0, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 6, 4, 7],
        [0, 6, 6, 4, 2, 7, 5, 7],
        [0, 6, 6, 5, 2, 7, 4, 7],
        [0, 6, 6, 5, 2, 6, 5, 7],
        [0, 6, 6, 5, 2, 6, 4, 7]], dtype=int64))

In [30]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1000)

In [31]:
d = 1
PBGDM_PBN_P4_d2 = np.array([[50 - d,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0,  0, 50 - d,  0,  0,      0], 
                            [     d,      d,      d,  0,      d,  0,  0,      d],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0, 50 - d, 50 - d,  0,      0, 25,  0,      0],
                            [     0,      0,      0,  0,      0, 25,  0, 50 - d]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P4_d2)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(11,
 [1, 1, 1, 1, 1, 10, 10, 12, 1, 1, 11],
 array([[3, 6, 6, 4, 2, 6, 4, 7],
        [0, 3, 6, 5, 2, 7, 5, 7],
        [0, 6, 3, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 3, 7, 5, 7],
        [0, 6, 6, 4, 2, 6, 4, 3],
        [0, 6, 6, 0, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 6, 4, 7],
        [0, 6, 6, 4, 2, 7, 5, 7],
        [0, 6, 6, 5, 2, 7, 4, 7],
        [0, 6, 6, 5, 2, 6, 5, 7],
        [0, 6, 6, 5, 2, 6, 4, 7]], dtype=int64))

In [32]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1009)

In [33]:
d = 3
PBGDM_PBN_P4_d3 = np.array([[100 - d,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0,  0, 100 - d,  0,  0,       0], 
                            [      d,       d,       d,  0,       d,  0,  0,       d],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0, 100 - d, 100 - d,  0,       0, 50,  0,       0],
                            [      0,       0,       0,  0,       0, 50,  0, 100 - d]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P4_d3)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(11,
 [3, 3, 3, 3, 3, 20, 20, 21, 3, 3, 18],
 array([[3, 6, 6, 4, 2, 6, 4, 7],
        [0, 3, 6, 5, 2, 7, 5, 7],
        [0, 6, 3, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 3, 7, 5, 7],
        [0, 6, 6, 4, 2, 6, 4, 3],
        [0, 6, 6, 0, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 6, 4, 7],
        [0, 6, 6, 4, 2, 7, 5, 7],
        [0, 6, 6, 5, 2, 7, 4, 7],
        [0, 6, 6, 5, 2, 6, 5, 7],
        [0, 6, 6, 5, 2, 6, 4, 7]], dtype=int64))

In [34]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=817)

In [35]:
d = 2
PBGDM_PBN_P4_d4 = np.array([[50 - d,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0,  0, 50 - d,  0,  0,      0], 
                            [     d,      d,      d,  0,      d,  0,  0,      d],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0, 50 - d, 50 - d,  0,      0, 25,  0,      0],
                            [     0,      0,      0,  0,      0, 25,  0, 50 - d]])

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P4_d4)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(11,
 [2, 2, 2, 2, 2, 9, 10, 9, 2, 3, 7],
 array([[3, 6, 6, 4, 2, 6, 4, 7],
        [0, 3, 6, 5, 2, 7, 5, 7],
        [0, 6, 3, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 3, 7, 5, 7],
        [0, 6, 6, 4, 2, 6, 4, 3],
        [0, 6, 6, 4, 2, 7, 5, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 6, 5, 7],
        [0, 6, 6, 5, 2, 7, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7],
        [0, 6, 6, 1, 2, 7, 4, 7]], dtype=int64))

In [36]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(0)

### 4. Fourth PBN matrix $P_6(d)$, where $d = 0.01, 0.02, 0.03, 0.04$.

In [37]:
PBGDM_PBN_P6_d1 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d1[0:8, 0:8] = PBGDM_PBN_P4_d1
PBGDM_PBN_P6_d1[8:16, 8:16] = PBGDM_PBN_P4_d1

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P6_d1)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(16,
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 20, 5, 20, 5, 20, 20],
 array([[ 3,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  3,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  3,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  3,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  3,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7, 11, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 11, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 11, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 11, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 11],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  4,  2,  7,  5,  7,  8, 14, 14, 12, 10, 15, 13, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,

In [38]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=2017)

In [39]:
PBGDM_PBN_P6_d2 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d2[0:8, 0:8] = PBGDM_PBN_P4_d2
PBGDM_PBN_P6_d2[8:16, 8:16] = PBGDM_PBN_P4_d2

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P6_d2)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(14,
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10],
 array([[ 3,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  3,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  3,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  3,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  3,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7, 11, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 11, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 11, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 11, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 11],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 1

In [40]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=1341)

In [41]:
PBGDM_PBN_P6_d3 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d3[0:8, 0:8] = PBGDM_PBN_P4_d3
PBGDM_PBN_P6_d3[8:16, 8:16] = PBGDM_PBN_P4_d3

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P6_d3)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(16,
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 17, 1, 17, 1, 17, 17],
 array([[ 3,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  3,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  3,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  3,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  3,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7, 11, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 11, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 11, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 11, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 11],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  4,  2,  7,  5,  7,  8, 14, 14, 12, 10, 15, 13, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,

In [42]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=2561)

In [43]:
PBGDM_PBN_P6_d4 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d4[0:8, 0:8] = PBGDM_PBN_P4_d4
PBGDM_PBN_P6_d4[8:16, 8:16] = PBGDM_PBN_P4_d4

time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(PBGDM_PBN_P6_d4)
time_after_execution = datetime.datetime.now()

(k, coefficient_list_xi, BN_matrices_list_Ai)

(16,
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 1, 7, 1, 7, 7],
 array([[ 3,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  3,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  3,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  3,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  3,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7, 11, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 11, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 11,  9, 10, 15, 13, 15],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 11, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 11],
        [ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  0,  2,  7,  5,  7,  8, 14, 14,  8, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  7,  5,  7,  8, 14, 14,  9,

In [44]:
execution_duration = time_after_execution - time_before_execution
execution_duration

datetime.timedelta(microseconds=2868)

# Random TPM Experiment

In [8]:
decompositions_lengths_list = []
execution_times_list = []
coefficients_lengths_list = []
A_array_shapes_list = []
output_eq_input_list = []
row_dim = 32
col_dim = 32

for k in range(4001, 6001):
    TPM_file_path = './random_pTPMs/dim32_pTPM_' + str(k) + '.npy'
    current_TPM = np.load(TPM_file_path)
    
    time_before_execution = datetime.datetime.now()
    length_k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(current_TPM)
    time_after_execution = datetime.datetime.now()
    execution_duration = time_after_execution - time_before_execution
    
    decompositions_lengths_list.append(length_k)
    execution_times_list.append(pd.Timedelta(execution_duration))
    coefficients_lengths_list.append(len(coefficient_list_xi))
    A_array_shapes_list.append(BN_matrices_list_Ai.shape)
    
    decomposition_sum = add_up_a_decomposition(coefficient_list_xi, BN_matrices_list_Ai, row_dim, col_dim)
    is_output_eq_input = np.array_equal(current_TPM, decomposition_sum)
    output_eq_input_list.append(is_output_eq_input)

[Sanity Check 1] Check whether decompositions_lengths_list and coefficients_lengths_list are identical.

In [9]:
decompositions_lengths_list == coefficients_lengths_list

True

[Sanity Check 2] Check whether the length of each element of A_array_shapes_list matches the corresponding length in decompositions_lengths_list.

In [10]:
A_array_lengths_list = [x[0] for x in A_array_shapes_list]
decompositions_lengths_list == A_array_lengths_list

True

[Sanity Check 3] Check whether each element of output_eq_input_list equals True.

In [11]:
all(output_eq_input_list)

True

Save the experimental data to a folder.

In [14]:
lengths_path = './expt_results/SER1_decom_lengths.json'
times_path = './expt_results/SER1_exe_time.pkl'
coefficients_path = './expt_results/SER1_coef_lengths.json'
A_array_shapes_path = './expt_results/SER1_A_array_shapes.json'
out_eq_in_path = './expt_results/SER1_out_eq_in.json'

with open(lengths_path, "w") as out_1:
    json.dump(decompositions_lengths_list, out_1)

execution_times_pdSeries = pd.Series(execution_times_list)
execution_times_pdSeries.to_pickle(times_path)

with open(coefficients_path, "w") as out_2:
    json.dump(coefficients_lengths_list, out_2)

with open(A_array_shapes_path, "w") as out_3:
    json.dump(A_array_shapes_list, out_3)
    
with open(out_eq_in_path, "w") as out_4:
    json.dump(output_eq_input_list, out_4)